In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
from torch.utils.data import  DataLoader
from handover_grasping.model import HANet
from handover_grasping.Datavisualizer import handover_grasping_dataset

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


DataLoader and parameters

In [3]:
BATCH_SIZE = 5
EPOCH = 50
SAVE_EVERY = 25
DATA_PATH = '/home/arg/handover_grasping/HANet_datasets'

dataset = handover_grasping_dataset(DATA_PATH)
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 8)

Initial HANet

In [2]:
net = HANet(4)
net = net.cuda()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /home/arg/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

Loss function and optimizer

In [4]:
criterion = nn.BCEWithLogitsLoss().cuda()

optimizer = optim.Adam(net.parameters(), lr = 1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 25, gamma = 0.1)

Start Training

In [11]:
if os.path.isdir(DATA_PATH + '/weight') == False:
    os.mkdir(DATA_PATH + '/weight')
    
loss_l = []
for epoch in range(EPOCH):
    loss_sum = 0.0
    ts = time.time()
    for i_batch, sampled_batched in enumerate(dataloader):
        print("\r[{:03.2f} %]".format(i_batch/float(len(dataloader))*100.0), end="\r")
        optimizer.zero_grad()
        color = sampled_batched['color'].cuda()
        depth = sampled_batched['depth'].cuda()
        label = sampled_batched['label'].permute(0,2,3,1).cuda().float()
        predict = net(color, depth)

        loss = criterion(predict, label)

        loss.backward()
        loss_sum += loss.detach().cpu().numpy()
        optimizer.step()
    scheduler.step()
    loss_l.append(loss_sum/len(dataloader))
    if (epoch+1)%SAVE_EVERY==0:
        torch.save(net.state_dict(), DATA_PATH + '/weight/grapnet_{}_{}.pth' .format(epoch+1, round(loss_l[-1],3)))

    print("Epoch: {}| Loss: {}| Time elasped: {}".format(epoch+1, loss_l[-1], time.time()-ts))